<a href="https://colab.research.google.com/github/Rjlee22/REU/blob/main/Stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow pandas scikit-learn
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import glob
from google.colab import drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Load preprocessed price data
# Verify the file path to ensure it's correct
file_paths = '/content/drive/My Drive/REU/preprocessed'
  # Add *.csv to match CSV files

data_list = []

for file_path in file_paths:
    df = pd.read_csv(file_path, delimiter='\t')
    data_list.append(df)

data = pd.concat(data_list, ignore_index=True)
data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


IsADirectoryError: [Errno 21] Is a directory: '/'

In [ ]:
# List files in a specific folder
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import TensorDataset, DataLoader
folder_path = '/content/drive/My Drive/REU/preprocessed'

txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

dfs = []

for txt_file in txt_files:
    file_path = os.path.join(folder_path, txt_file)
    df = pd.read_csv(file_path, delimiter='\t', header=None)  # Adjust delimiter and header as needed
    dfs.append(df)

data = pd.concat(dfs, ignore_index=True)

data = data.drop(data.columns[0], axis=1)

y = data.iloc[:, 1]

# Extract features by dropping the target variable
X = data.drop(data.columns[1], axis=1)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)




In [ ]:
import torch.nn as nn
import torch.optim as optim
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

input_size = X_train.shape[1]
model = SimpleNN(input_size).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training the model
num_epochs = 25
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * X_batch.size(0)

    train_loss /= len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}")

Epoch 1/25, Loss: 0.0029
Epoch 2/25, Loss: 0.0001
Epoch 3/25, Loss: 0.0000
Epoch 4/25, Loss: 0.0000
Epoch 5/25, Loss: 0.0000
Epoch 6/25, Loss: 0.0000
Epoch 7/25, Loss: 0.0000
Epoch 8/25, Loss: 0.0000
Epoch 9/25, Loss: 0.0000
Epoch 10/25, Loss: 0.0000
Epoch 11/25, Loss: 0.0000
Epoch 12/25, Loss: 0.0000
Epoch 13/25, Loss: 0.0000
Epoch 14/25, Loss: 0.0000
Epoch 15/25, Loss: 0.0000
Epoch 16/25, Loss: 0.0000
Epoch 17/25, Loss: 0.0000
Epoch 18/25, Loss: 0.0000
Epoch 19/25, Loss: 0.0000
Epoch 20/25, Loss: 0.0000
Epoch 21/25, Loss: 0.0000
Epoch 22/25, Loss: 0.0000
Epoch 23/25, Loss: 0.0000
Epoch 24/25, Loss: 0.0000
Epoch 25/25, Loss: 0.0000


In [ ]:
model.eval()
test_loss = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch).squeeze()
        test_loss += criterion(outputs, y_batch).item() * X_batch.size(0)
        all_preds.append(outputs.cpu().numpy())
        all_labels.append(y_batch.cpu().numpy())

test_loss /= len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f}")

Test Loss: 0.0000


In [ ]:
all_preds

array([ 0.05417788,  0.0525111 ,  0.00141452, ...,  0.04249628,
       -0.00100607,  0.03147447], dtype=float32)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
#all_preds = np.concatenate(all_preds)
#all_labels = np.concatenate(all_labels)

# For simplicity, we'll assume binary classification with a threshold of 0.5
threshold = 0.5
binary_preds = (all_preds > threshold).astype(int)
binary_labels = (all_labels > threshold).astype(int)

# Calculate accuracy and F1 score
accuracy = accuracy_score(binary_labels, binary_preds)
f1 = f1_score(binary_labels, binary_preds)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.9999
F1 Score: 0.8571
